# Lab 12 – Seq2seq with Attention for Machine Translation
- We are going to to preprocess data from **Multi30k**, a well-known dataset containing sentences in both German and English using `torchtext` and use it to train a **sequence-to-sequence model with attention** that can translate German sentences into English.
- It is based on [this](https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html) tutorial which was inspired from a PyTorch community member, Ben Trevett.
- Enable GPU for this lab.
- Steps
  - Preprocessing using spacy tokeniser
  - DataLoader
  - Seq2seq model with Attention
  - Training & Evaluation 

### Download required tokeniser

In [1]:
!python -m spacy download de

     |████████████████████████████████| 14.9 MB 4.3 MB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.2.5-py3-none-any.whl size=14907055 sha256=335c09ee79b4add0f63f5461f79af684a649bc2a5fb899d86a0eaaa3c2cee840
  Stored in directory: /tmp/pip-ephem-wheel-cache-azcvz7el/wheels/00/66/69/cb6c921610087d2cab339062345098e30a5ceb665360e7b32a
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


**IMPORTANT** Please `Restart runtime` before continuing.

In [2]:
from torchtext.datasets import Multi30k

from torchtext.data.utils import get_tokenizer

import os

import spacy

### Load the German and English tokenisers
If the code cell below does not work, install the following, Restart Runtime and try again
- `!pip install spacy`
- `!python -m spacy download en`


In [3]:
de_tokenizer = get_tokenizer('spacy', language='de')
en_tokenizer = get_tokenizer('spacy', language='en')

### Load the Dataset

In [4]:
train_iter, valid_iter, test_iter = Multi30k(root='./data', split=('train', 'valid', 'test'), language_pair=('de', 'en'))

100%|██████████| 1.21M/1.21M [00:02<00:00, 527kB/s]
100%|██████████| 46.3k/46.3k [00:00<00:00, 90.7kB/s]
100%|██████████| 43.9k/43.9k [00:00<00:00, 86.0kB/s]


### View sample sentences

In [5]:
src_sentence, tgt_sentence = next(valid_iter)
print(src_sentence, tgt_sentence)

# Q1. Determine how many items (sentences) are in the train, valid and test sets
  ## ANSWER: train: 1.21M items / valid: 46.3k items / test: 43.9k items

Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen
 A group of men are loading cotton onto a truck



In [ ]:
# Q2. What is the relationship between a source and target sentence (src_sentence and tgt_sentence)?
  ## ANSWER: tgt sentence is a translated sentence of source sentence (German -> English)

### Specify the directories of the data

In [6]:
root_dir = "./data/Multi30k"

train_path_de = os.path.join(root_dir, "train.de")
train_path_en = os.path.join(root_dir, "train.en")

valid_path_de = os.path.join(root_dir, "val.de")
valid_path_en = os.path.join(root_dir, "val.en")

test_path_de = os.path.join(root_dir, "test.de")
test_path_en = os.path.join(root_dir, "test.en")



### Create German and English Vocabs
- We are going to use `build_vocab_from_iterator()` which we have used in Lab 10

In [18]:
import io

from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
from torchtext.vocab import Vocab


def yield_tokens(file_path):
  with io.open(file_path, encoding = 'utf-8') as f:
    for line in f:
      yield line.strip().split()

de_vocab = build_vocab_from_iterator(yield_tokens(train_path_de), specials=['<unk>', '<pad>', '<sos>', '<eos>'])
de_vocab.set_default_index(de_vocab["<unk>"])

en_vocab = build_vocab_from_iterator(yield_tokens(train_path_en), specials=['<unk>', '<pad>', '<sos>', '<eos>'])
en_vocab.set_default_index(de_vocab["<unk>"])

print("The length of the de_vocab is", len(de_vocab))

de_stoi = de_vocab.get_stoi()
print("The DE index of '<unk>' is", de_stoi['<unk>'])
print(de_stoi)

de_itos = de_vocab.get_itos()
print("The DE token at index 2 is", de_itos[2])
print(de_itos)

print("The length of the en_vocab is", len(en_vocab))

en_stoi = en_vocab.get_stoi()
print("The EN index of '<unk>' is", en_stoi['<unk>'])
print(en_stoi)

en_itos = en_vocab.get_itos()
print("The EN token at index 4 is", en_itos[4])
print(en_itos)


The length of the de_vocab is 24893
The DE index of '<unk>' is 0
{'„nackter': 24887, '„moose': 24886, '„keep': 24883, '„freies': 24880, '„eXXXotica': 24877, '„Your': 24872, '„Vote“': 24868, '„Volunteer': 24867, '„Twister“': 24866, '„Toys': 24865, '„To': 24863, '„Tickets': 24861, '„Teva': 24859, "„Terrible's“": 24858, '„Stoppt': 24855, '„SAG': 24843, '„Rock': 24841, '„RONA“': 24836, '„O“': 24825, '„Open“-Anzeige.': 24823, '„Ohio': 24822, '„Obama': 24819, '„November': 24816, '„Niederschlagen“': 24814, '„Miami': 24811, '„Mayores“': 24808, '„Love': 24806, '„Live': 24804, '„Le': 24803, '„Koffee': 24799, '„Kenji“': 24798, '„Keep': 24797, '„Jobs': 24795, '„Incredible': 24792, '„Impeach“': 24791, '„Hello': 24786, '„Hair': 24785, '„Grand': 24782, '„Gidday“': 24779, '„Freiheitsstatuenhaltung“': 24776, '„Free': 24775, '„Fabrik.“': 24771, '„Exodus“': 24770, '„Energizer“-Hasenohren': 24768, '„Elmo“': 24767, '„Ein': 24765, '„END': 24764, '„ELOIN“': 24763, '„Connie': 24758, '„Cigar': 24756, '„Car': 2

In [ ]:
# Q3. Why are the lengths of de_vocab and en_vocab different?
  ## Because German and English have diffrent forms of language.

# Q4. Modify build_vocab_from_iterator to only include words that appear at least 2 times in the whole corpus. Rerun the code.

# Q5. Are there more or less tokens now? Why?
  ## There are less tokens. That's because the cutoff has been harsher than before(when min_freq=0)

# Q6. What are the tokens <unk>, <pad>, <sos> and <eos> used for?
  ## <unk>: It is used for unknown, rare words that did not fit the vocab.
  ## <pad>: It is used for padding. For example, for fixing the length of various sentences.
  ## <sos>: It is used for the first token of sentence.
  ## <eos>: It is used for the last token of sentence.

# Q7. What do the methods get_stoi() and get_itos() do?
  ## get_stoi() is for getting index that matches the given token as an argument within vocab / 
  ## get_itos() is for getting word token that matches the given index as an argument within vocab.

### Convert the Datasets into Tensors

In [8]:
import torch
# import io

def data_process(path_de, path_en):
  raw_de_iter = iter(io.open(path_de, encoding="utf8"))
  raw_en_iter = iter(io.open(path_en, encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                            dtype=torch.long)
    data.append((de_tensor_, en_tensor_))
  return data

train_data = data_process(train_path_de, train_path_en)
val_data = data_process(valid_path_de, valid_path_en)
test_data = data_process(test_path_de, test_path_en)

In [ ]:
# Q8. Can you think of ONE other preprocessing that could be done on the this dataset?
  ## We can give stop words to eliminate meaningless tokens.

### DataLoader
- Generate batches by appending `<sos>` at the start of sentence and `<eos>` at the end of the sentence
- Pad shorter sentences by calling `pad_sequence()`

In [9]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
SOS_IDX = de_vocab['<sos>']
EOS_IDX = de_vocab['<eos>']


def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([SOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([SOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

## NN Module and Optimiser
### Encoder
- A bidirectional GRU is used

In [10]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor


class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()

        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        # Embedding layer 
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # GRU
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        # Fully connected layer
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        #src = [src len, batch size]

        embedded = self.dropout(self.embedding(src))

        #embedded = [src len, batch size, emb dim]

        # Forward pass through GRU
        outputs, hidden = self.rnn(embedded)

        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        # encoder RNNs fed through a linear layer

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden


### Attention
- Takes in the previous hidden state of the decoder, $s_{t-1}$, and all of the stacked forward and backward hidden states from the encoder, $H$. The layer will output an attention vector, $a_t$, that is the length of the source sentence, each element is between 0 and 1 and the entire vector sums to 1.
- This layer takes what we have decoded so far, $s_{t-1}$, and all of what we have encoded, $H$, to produce a vector, $a_t$, that represents which words in the source sentence we should pay the most attention to in order to correctly predict the next word to decode, $\hat{y}_{t+1}$.
- Calculates the energy between the previous decoder hidden state and the encoder hidden states. As our encoder hidden states are a sequence of $T$ tensors, and our previous decoder hidden state is a single tensor, the first thing we do is repeat the previous decoder hidden state $T$ times. We then calculate the energy (attention/alignment score), $E_t$, between them by concatenating them together and passing them through a linear layer (attn) and a $\tanh$ activation function. This can be thought of as **how well each encoder hidden state "matches" the previous decoder hidden state.**
- We can think of $v$ as the weights for a weighted sum of the energy across all encoder hidden states. These weights tell us how much we should attend to each token in the source sequence. The parameters of $v$ are initialized randomly, but learned with the rest of the model via backpropagation. Note how $v$ is not dependent on time, and the same $v$ is used for each time-step of the decoding. Implement $v$ as a linear layer without a bias.


In [11]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        # v
        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        # Energy = attention_score (e_ij), v = self.attn (Linear layer)
        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        # This gives us the attention over the source sentence
        return F.softmax(attention, dim=1)


### Decoder
- The decoder contains the attention layer, attention, which takes the previous hidden state, $s_{t-1}$, all of the encoder hidden states, $H$, and returns the attention vector, $a_t$.
- We then use this attention vector to create a weighted source vector, $w_t$, denoted by weighted, which is a weighted sum of the encoder hidden states, $H$, using $a_t$ as the weights.


In [12]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        #a = [batch size, src len]

        a = a.unsqueeze(1)

        #a = [batch size, 1, src len]

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        #encoder_outputs = [batch size, src len, enc hid dim * 2]

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]

        #input = [1, batch size]
        input = input.unsqueeze(0)


        #embedded = [1, batch size, emb dim]
        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        #output = [batch size, output dim]
        return output, decoder_hidden.squeeze(0)




### Seq2Seq
- Added something called **teacher forcing** - the ground truth label at current time step is used as input for the next time step
- Briefly going over all of the steps:
  - the outputs tensor is created to hold all predictions, $\hat{Y}$
  - the source sequence, $X$, is fed into the encoder to receive $z$ and $H$
  - the initial decoder hidden state is set to be the context vector, $s_0 = z = h_T$
  - we use a batch of <sos> tokens as the first input, $y_1$
  - we then decode within a loop:
    - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and all encoder outputs, $H$, into the decoder
    - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
    - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(output, hidden, encoder_outputs)

            #place predictions in a tensor holding predictions for each token
            outputs[t] = output

            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio

            #get the highest predicted token from our predictions
            top1 = output.max(1)[1]

            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            output = (trg[t] if teacher_force else top1)

        return outputs

### Create Model Instance and Optimiser
- This model follows the architecture described [here](https://arxiv.org/abs/1409.0473)
- You can find a significantly more commented version [here](https://github.com/SethHWeidman/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb).
- Weights initialisation is introduced. We will initialize all biases to zero and all weights from $\mathcal{N}(0, 0.01)$.
- **Note:** this model is just an example model that can be used for language translation; we choose it because it is a standard model for the task, not because it is the recommended model to use for translation. 
- State-of-the-art models are currently based on Transformers; you can see PyTorch's capabilities for implementing Transformer layers [here](https://pytorch.org/docs/stable/nn.html#transformer-layers); and in particular, the "attention" used in the model below is different from the *multi-headed self-attention* present in a transformer model.

In [14]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)


# Initialise weights
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01) # Initialise weights with values drawn from the normal distribution
        else:
            nn.init.constant_(param.data, 0) # Initialiase biases to 0


print(model.apply(init_weights))

optimizer = optim.Adam(model.parameters())

# Determine number of paramters
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(9762, 32)
    (rnn): GRU(32, 64, bidirectional=True)
    (fc): Linear(in_features=128, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=192, out_features=8, bias=True)
    )
    (embedding): Embedding(7964, 32)
    (rnn): GRU(160, 64)
    (out): Linear(in_features=224, out_features=7964, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)
The model has 2,449,956 trainable parameters


### Loss Function
**Note:** When scoring the performance of a language translation model in particular, we have to tell the `nn.CrossEntropyLoss` function to ignore the indices where the target is simply padding.

In [15]:
PAD_IDX = en_stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

### Train and Evaluate Functions

In [16]:
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


### Let's Train!
- Added `perplexity` which is a common metrics for evaluating language models. We can add perplexity by exponentiating the loss.
- Training takes around 10 minutes (1 min per epoch)

In [17]:
import math

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 's2s-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Perplexity: {math.exp(train_loss):7.3f} |')
    print(f'\t Val. Loss: {valid_loss:.3f} |   Val Perplexity: {math.exp(valid_loss):7.3f} |')


Epoch: 01 | Time: 0m 27s
	Train Loss: 5.453 | Train Perplexity: 233.358 |
	 Val. Loss: 4.789 |   Val Perplexity: 120.138 |
Epoch: 02 | Time: 0m 27s
	Train Loss: 4.532 | Train Perplexity:  92.908 |
	 Val. Loss: 4.570 |   Val Perplexity:  96.538 |
Epoch: 03 | Time: 0m 27s
	Train Loss: 4.301 | Train Perplexity:  73.766 |
	 Val. Loss: 4.448 |   Val Perplexity:  85.418 |
Epoch: 04 | Time: 0m 27s
	Train Loss: 4.162 | Train Perplexity:  64.213 |
	 Val. Loss: 4.371 |   Val Perplexity:  79.107 |
Epoch: 05 | Time: 0m 27s
	Train Loss: 4.072 | Train Perplexity:  58.656 |
	 Val. Loss: 4.331 |   Val Perplexity:  75.985 |
Epoch: 06 | Time: 0m 27s
	Train Loss: 4.020 | Train Perplexity:  55.696 |
	 Val. Loss: 4.295 |   Val Perplexity:  73.344 |
Epoch: 07 | Time: 0m 27s
	Train Loss: 3.962 | Train Perplexity:  52.538 |
	 Val. Loss: 4.257 |   Val Perplexity:  70.575 |
Epoch: 08 | Time: 0m 27s
	Train Loss: 3.891 | Train Perplexity:  48.970 |
	 Val. Loss: 4.203 |   Val Perplexity:  66.910 |
Epoch: 09 | Time

### Test the model on the test set using these "best" parameters

In [19]:
model.load_state_dict(torch.load('s2s-model.pt'))

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Perplexity: {math.exp(test_loss):7.3f} |')

| Test Loss: 4.139 | Test Perplexity:  62.723 |


See Ben Trevett's tutorial [here](https://github.com/bentrevett/pytorch-seq2seq/blob/master/3%20-%20Neural%20Machine%20Translation%20by%20Jointly%20Learning%20to%20Align%20and%20Translate.ipynb) for a better explanation of the steps taken for this model (it uses older code).